In [1]:
# Import our dependencies

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.

import pandas as pd 
orig_application_df = pd.read_csv("Resources/charity_data.csv")
orig_application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'

application_df = orig_application_df.drop(columns=['EIN', 'NAME'], axis=1)
application_df.head()

,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [3]:
# Determine the number of unique values in each column

application_df.nunique()

APPLICATION_TYPE            17
AFFILIATION                  6
CLASSIFICATION              71
USE_CASE                     5
ORGANIZATION                 4
STATUS                       2
INCOME_AMT                   9
SPECIAL_CONSIDERATIONS       2
ASK_AMT                   8747
IS_SUCCESSFUL                2
dtype: int64

In [4]:
# Look at APPLICATION_TYPE value counts for binning

application_df['APPLICATION_TYPE'].value_counts()

T3     27037
T4      1542
T6      1216
T5      1173
T19     1065
T8       737
T7       725
T10      528
T9       156
T13       66
T12       27
T2        16
T25        3
T14        3
T15        2
T29        2
T17        1
Name: APPLICATION_TYPE, dtype: int64

In [5]:
# Choose a cutoff value (500) and create a list of names to be replaced
# use the variable name `application_types_to_replace`

application_counts = application_df['APPLICATION_TYPE'].value_counts().to_frame().reset_index()

app_counts = application_counts.loc[application_counts['APPLICATION_TYPE'] < 500]    
application_types_to_replace = app_counts['index'].tolist()

In [6]:
# Replace in dataframe

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app, "Other")

# Check to make sure binning was successful

application_df['APPLICATION_TYPE'].value_counts()

T3       27037
T4        1542
T6        1216
T5        1173
T19       1065
T8         737
T7         725
T10        528
Other      276
Name: APPLICATION_TYPE, dtype: int64

In [7]:
# Look at CLASSIFICATION value counts for binning

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
         ...  
C6100        1
C1248        1
C1245        1
C2600        1
C2500        1
Name: CLASSIFICATION, Length: 71, dtype: int64

In [8]:
# You may find it helpful to look at CLASSIFICATION value counts >1

application_df['CLASSIFICATION'].value_counts().loc[lambda x : x > 1 ]

C1000    17326
C2000     6074
C1200     4837
C3000     1918
C2100     1883
C7000      777
C1700      287
C4000      194
C5000      116
C1270      114
C2700      104
C2800       95
C7100       75
C1300       58
C1280       50
C1230       36
C1400       34
C7200       32
C2300       32
C1240       30
C8000       20
C7120       18
C1500       16
C6000       15
C1800       15
C1250       14
C8200       11
C1278       10
C1238       10
C1237        9
C1235        9
C7210        7
C4100        6
C1720        6
C2400        6
C1600        5
C1257        5
C2710        3
C1260        3
C0           3
C1256        2
C1267        2
C3200        2
C1246        2
C1234        2
Name: CLASSIFICATION, dtype: int64

In [9]:
# Choose a cutoff value (1000) and create a list of classifications to be replaced
# use the variable name `classifications_to_replace`

classification_counts = application_df['CLASSIFICATION'].value_counts().to_frame().reset_index()

class_counts = classification_counts.loc[classification_counts['CLASSIFICATION'] < 1000]    
classifications_to_replace = class_counts['index'].tolist()

In [10]:
# Replace in dataframe

for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls, "Other")
    
# Check to make sure binning was successful

application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     2261
C3000     1918
C2100     1883
Name: CLASSIFICATION, dtype: int64

In [11]:
# Convert categorical data to numeric with `pd.get_dummies`

application_converted = pd.get_dummies(application_df, prefix=['APPLICATION_TYPE', 'AFFILIATION','CLASSIFICATION','USE_CASE','ORGANIZATION','INCOME_AMT','SPECIAL_CONSIDERATIONS'])
application_converted.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
1,1,108590,1,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
4,1,142590,1,0,0,0,1,0,0,0,...,0,0,1,0,0,0,0,0,1,0


In [12]:
# Split our preprocessed data into our features and target arrays

X = application_converted.drop(columns='IS_SUCCESSFUL', axis=1)
y = application_converted['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [13]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

OPTIMIZATION 1: WITHOUT REMOVING OUTLIERS

In [14]:
# Define the model

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=60, activation="relu", input_dim=43))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 60)                2640      
_________________________________________________________________
dense_1 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 31        
Total params: 4,501
Trainable params: 4,501
Non-trainable params: 0
_________________________________________________________________


In [15]:
# Compile the model

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [16]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
804/804 [==============================] - 1s 911us/step - loss: 0.5723 - accuracy: 0.7221
Epoch 2/100
804/804 [==============================] - 1s 726us/step - loss: 0.5560 - accuracy: 0.7283
Epoch 3/100
804/804 [==============================] - 1s 728us/step - loss: 0.5521 - accuracy: 0.7304
Epoch 4/100
804/804 [==============================] - 1s 727us/step - loss: 0.5506 - accuracy: 0.7293
Epoch 5/100
804/804 [==============================] - 1s 753us/step - loss: 0.5492 - accuracy: 0.7312
Epoch 6/100
804/804 [==============================] - 1s 768us/step - loss: 0.5487 - accuracy: 0.7315
Epoch 7/100
804/804 [==============================] - 1s 735us/step - loss: 0.5480 - accuracy: 0.7328
Epoch 8/100
804/804 [==============================] - 1s 762us/step - loss: 0.5472 - accuracy: 0.73220s - loss: 0.5
Epoch 9/100
804/804 [==============================] - 1s 748us/step - loss: 0.5471 - accuracy: 0.7322
Epoch 10/100
804/804 [==============================] - 1s 

In [17]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5594 - accuracy: 0.7279
Loss: 0.5593575239181519, Accuracy: 0.7279300093650818


OPTIMIZATION 2: REMOVING OUTLIERS

In [18]:
# Removing outliers from ASK_AMT

# Create quantile to set bounds for outlier to be established

Q1 = application_converted['ASK_AMT'].quantile(0.25)
Q3 = application_converted['ASK_AMT'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1-(1.5*IQR)
upper_bound = Q3+(1.5*IQR)

application_outliers_filtered = application_converted.loc[application_converted['ASK_AMT'] < upper_bound]
application_outliers_filtered.head()

,STATUS,ASK_AMT,IS_SUCCESSFUL,APPLICATION_TYPE_Other,APPLICATION_TYPE_T10,APPLICATION_TYPE_T19,APPLICATION_TYPE_T3,APPLICATION_TYPE_T4,APPLICATION_TYPE_T5,APPLICATION_TYPE_T6,...,INCOME_AMT_1-9999,INCOME_AMT_10000-24999,INCOME_AMT_100000-499999,INCOME_AMT_10M-50M,INCOME_AMT_1M-5M,INCOME_AMT_25000-99999,INCOME_AMT_50M+,INCOME_AMT_5M-10M,SPECIAL_CONSIDERATIONS_N,SPECIAL_CONSIDERATIONS_Y
0,1,5000,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0
2,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0
3,1,6692,1,0,0,0,1,0,0,0,...,0,1,0,0,0,0,0,0,1,0
5,1,5000,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,0
9,1,5000,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,1,0


In [19]:
# Split our preprocessed data into our features and target arrays

X = application_outliers_filtered.drop(columns='IS_SUCCESSFUL', axis=1)
y = application_outliers_filtered['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [20]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [21]:
# Define the model

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=60, activation="relu", input_dim=43))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 60)                2640      
_________________________________________________________________
dense_4 (Dense)              (None, 30)                1830      
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 31        
Total params: 4,501
Trainable params: 4,501
Non-trainable params: 0
_________________________________________________________________


In [22]:
# Compile the model

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [23]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=100)

Epoch 1/100
612/612 [==============================] - 1s 957us/step - loss: 0.5509 - accuracy: 0.7385
Epoch 2/100
612/612 [==============================] - 1s 1ms/step - loss: 0.5340 - accuracy: 0.7460
Epoch 3/100
612/612 [==============================] - 0s 805us/step - loss: 0.5328 - accuracy: 0.7481
Epoch 4/100
612/612 [==============================] - 1s 911us/step - loss: 0.5305 - accuracy: 0.7485
Epoch 5/100
612/612 [==============================] - 1s 832us/step - loss: 0.5292 - accuracy: 0.7473
Epoch 6/100
612/612 [==============================] - 1s 839us/step - loss: 0.5279 - accuracy: 0.7500
Epoch 7/100
612/612 [==============================] - 0s 743us/step - loss: 0.5279 - accuracy: 0.7481
Epoch 8/100
612/612 [==============================] - 0s 741us/step - loss: 0.5266 - accuracy: 0.7499
Epoch 9/100
612/612 [==============================] - 1s 821us/step - loss: 0.5269 - accuracy: 0.7487
Epoch 10/100
612/612 [==============================] - 1s 840us/step - los

In [24]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 0s - loss: 0.5522 - accuracy: 0.7437
Loss: 0.5521746277809143, Accuracy: 0.7437155246734619


OPTIMIZATION 3: CHANGING PARAMETERS

In [25]:
# Split our preprocessed data into our features and target arrays

X = application_outliers_filtered.drop(columns='IS_SUCCESSFUL', axis=1)
y = application_outliers_filtered['IS_SUCCESSFUL']

# Split the preprocessed data into a training and testing dataset

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [26]:
# Create a StandardScaler instances

scaler = StandardScaler()

# Fit the StandardScaler

X_scaler = scaler.fit(X_train)

# Scale the data

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [27]:
# Define the model

nn = tf.keras.models.Sequential()

# First hidden layer

nn.add(tf.keras.layers.Dense(units=80, activation="relu", input_dim=43))

# Second hidden layer

nn.add(tf.keras.layers.Dense(units=30, activation="relu"))

# Output layer

nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model

nn.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 80)                3520      
_________________________________________________________________
dense_7 (Dense)              (None, 30)                2430      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 31        
Total params: 5,981
Trainable params: 5,981
Non-trainable params: 0
_________________________________________________________________


In [28]:
# Compile the model

nn.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [29]:
# Train the model

fit_model = nn.fit(X_train_scaled, y_train, epochs=50)

Epoch 1/50
612/612 [==============================] - 1s 717us/step - loss: 0.5526 - accuracy: 0.7388
Epoch 2/50
612/612 [==============================] - 0s 805us/step - loss: 0.5343 - accuracy: 0.7456
Epoch 3/50
612/612 [==============================] - 1s 841us/step - loss: 0.5317 - accuracy: 0.7473
Epoch 4/50
612/612 [==============================] - 0s 774us/step - loss: 0.5312 - accuracy: 0.7490
Epoch 5/50
612/612 [==============================] - 0s 714us/step - loss: 0.5285 - accuracy: 0.7500
Epoch 6/50
612/612 [==============================] - 0s 715us/step - loss: 0.5292 - accuracy: 0.7477
Epoch 7/50
612/612 [==============================] - 0s 728us/step - loss: 0.5270 - accuracy: 0.7489
Epoch 8/50
612/612 [==============================] - 0s 723us/step - loss: 0.5276 - accuracy: 0.7491
Epoch 9/50
612/612 [==============================] - 0s 795us/step - loss: 0.5262 - accuracy: 0.7507
Epoch 10/50
612/612 [==============================] - 0s 720us/step - loss: 0.526

In [30]:
# Evaluate the model using the test data

model_loss, model_accuracy = nn.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

204/204 - 0s - loss: 0.5429 - accuracy: 0.7474
Loss: 0.542873203754425, Accuracy: 0.7473942637443542


In [31]:
# Export our model to HDF5 file
 
nn.save('Models/AlphabetSoupCharity_Optimization.h5')